# Maestro Test Generator with CrewAI and Gemini

## Introduction
This notebook demonstrates how to build an automated test script generator that converts written test cases into Maestro test scripts. This approach allows QA teams to write test cases in natural language and automatically convert them to executable Maestro scripts.

In [7]:
## Setup and Dependencies

# Step 1: Install required packages
!pip install typing_extensions==4.7.1
!pip install pydantic==1.10.8  # Using older version for Python 3.10 compatibility
!pip install crewai==0.28.0 langchain_google_genai==0.0.6 pillow pyyaml ipywidgets

  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
altair 5.5.0 requires typing-extensions>=4.10.0; python_version < "3.14", but you have typing-extensions 4.7.1 which is incompatible.
fastapi 0.115.9 requires typing-extensions>=4.8.0, but you have typing-extensions 4.7.1 which is incompatible.
google-generativeai 0.8.3 requires google-ai-generativelanguage==0.6.10, but you have google-ai-generativelanguage 0.6.17 which is incompatible.
langchain 0.3.12 requires async-timeout<5.0.0,>=4.0.0; python_version < "3.11", but you have async-timeout 5.0.1 which is incompatible.
openai 1.74.0 requires typing-extensions<5,>=4.11, but you have typing-extensions 4.7.1 which is inc

## Import Necessary Libraries
These libraries provide the core functionality for our test generation system.

In [8]:
# Step 2: Import necessary libraries
import os
import json
import yaml
from typing import Dict, List, Optional
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets

from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import BaseTool

ImportError: cannot import name 'CrewAgentExecutor' from 'crewai.agents' (/usr/local/lib/python3.10/dist-packages/crewai/agents/__init__.py)

## API Setup
For this notebook to work, you need to have a Google API key with access to Gemini models.

In [ ]:
# Step 3: Setup credentials
# For Kaggle, use secrets or set environment variables
# You need to add your Google API key to Kaggle secrets
# Go to Kaggle → Account → API → Add new secret
# Name: GOOGLE_API_KEY, Value: your-api-key

# Get API key from Kaggle secrets
import kaggle.UserSecretsClient
user_secrets = kaggle.UserSecretsClient()
os.environ["GOOGLE_API_KEY"] = user_secrets.get_secret("GOOGLE_API_KEY")

## Helper Functions
These functions make it easier to display different types of content in the notebook.

In [ ]:
# Step 4: Display helper functions for the notebook
def display_code(code, language="yaml"):
    """Display code with syntax highlighting"""
    from IPython.display import display, Markdown
    md_content = f"```{language}\n{code}\n```"
    display(Markdown(md_content))

def display_json(json_data):
    """Pretty print JSON data"""
    if isinstance(json_data, str):
        json_data = json.loads(json_data)
    from IPython.display import JSON
    display(JSON(json_data))

## LLM Setup
We'll use Google's Gemini model for natural language processing and understanding.


In [ ]:
# Step 5: Setup the LLM (Gemini 2.5)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  # Using Gemini 1.5 Pro until 2.5 is available in the API
    temperature=0.2,
    convert_system_message_to_human=True,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

## Custom Tools
The following tools allow our agents to perform specialized tasks like screenshot analysis
and Maestro command lookup.

In [ ]:
# Step 6: Define tools needed by agents
class TestCaseParserTool(BaseTool):
    name = "test_case_parser"
    description = "Parses written test cases into structured format"
    
    def _run(self, test_cases_text: str) -> str:
        """Parse test cases from text into structured format"""
        print("Test Case Parser: Extracting test steps from written test cases...")
        
        # In a real implementation, you'd use the LLM to extract test steps
        # For this demo, we'll use a simplified approach
        
        # Parse the test cases into steps
        test_steps = []
        
        # Split by lines and look for numbered steps or bullet points
        lines = test_cases_text.strip().split('\n')
        current_step = None
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            # Check if this is a new step (starts with number or bullet)
            if line[0].isdigit() or line.startswith('- ') or line.startswith('* '):
                if current_step:
                    test_steps.append(current_step)
                current_step = {"description": line, "actions": []}
            elif current_step:
                # This is a sub-step or additional info for the current step
                current_step["actions"].append(line)
        
        # Add the last step if there is one
        if current_step:
            test_steps.append(current_step)
        
        return json.dumps(test_steps)

class MaestroDocumentationTool(BaseTool):
    name = "maestro_docs"
    description = "Retrieves relevant Maestro documentation for test actions"
    
    def _run(self, test_steps: str) -> str:
        """Find relevant Maestro commands for the test steps"""
        steps = json.loads(test_steps)
        commands = {}
        
        print("Documentation Agent: Finding Maestro commands for test steps...")
        
        # Common actions to Maestro command mapping
        action_mapping = {
            "tap": "tapOn",
            "click": "tapOn",
            "press": "tapOn",
            "input": "inputText",
            "type": "inputText",
            "enter": "inputText",
            "wait": "wait",
            "verify": "assertVisible",
            "check": "assertVisible",
            "assert": "assertVisible",
            "scroll": "scroll",
            "swipe": "swipe",
            "back": "pressBack"
        }
        
        # Process each step to find relevant commands
        for step in steps:
            step_text = step["description"].lower()
            actions = step.get("actions", [])
            
            for action_key, command in action_mapping.items():
                if action_key in step_text:
                    commands[step["description"]] = {
                        "command": command,
                        "syntax": f"{command}: \"relevant_element\"",
                        "description": f"Performs the {action_key} action as described in the test step."
                    }
                    break
        
        return json.dumps(commands)

class ValidateTestTool(BaseTool):
    name = "validate_test"
    description = "Validates a Maestro test flow for correctness"
    
    def _run(self, test_flow: str) -> str:
        """Check if the test flow is valid and correct issues"""
        print("Validation Agent: Checking test flow for correctness...")
        
        issues = []
        
        # Simple validation checks
        if "appId:" not in test_flow:
            issues.append("Missing appId in test flow")
        
        if "- runFlow:" not in test_flow and "- tapOn:" not in test_flow:
            issues.append("No test steps found in flow")
        
        if issues:
            print(f"❌ Validation found issues: {issues}")
            return json.dumps({"valid": False, "issues": issues})
        else:
            print("✅ Test flow validated successfully!")
            return json.dumps({"valid": True})

## Agent Definition
Each agent has a specialized role in the test generation process.

In [ ]:
# Step 7: Define agents
parser_agent = Agent(
    role="Test Case Parser",
    goal="Extract structured test steps from written test cases",
    backstory="You are an expert in understanding test case documentation and extracting actionable test steps from written requirements.",
    verbose=True,
    allow_delegation=True,
    tools=[TestCaseParserTool()],
    llm=llm
)

docs_agent = Agent(
    role="Maestro Documentation Expert",
    goal="Find the most appropriate Maestro commands for test steps",
    backstory="You've memorized the entire Maestro documentation and can quickly retrieve the right commands for any test action.",
    verbose=True,
    allow_delegation=True,
    tools=[MaestroDocumentationTool()],
    llm=llm
)

test_gen_agent = Agent(
    role="Test Script Generator",
    goal="Create comprehensive Maestro test flows that implement the test cases",
    backstory="You're a master test engineer who can craft elegant and robust test scripts for any mobile app.",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

validation_agent = Agent(
    role="Test Quality Assurance",
    goal="Ensure all generated tests are valid, efficient, and follow best practices",
    backstory="You have an eye for detail and can spot even the smallest issues in test scripts, ensuring they run correctly.",
    verbose=True,
    allow_delegation=True,
    tools=[ValidateTestTool()],
    llm=llm
)

## Task Definition
These tasks form a pipeline for processing screenshots and generating tests.

In [ ]:
# Step 8: Define tasks
def create_tasks(app_name, test_cases_text):
    parse_test_cases_task = Task(
        description=f"Parse the written test cases for {app_name} and extract structured test steps.",
        expected_output="A detailed JSON list of test steps with their descriptions and actions.",
        agent=parser_agent,
        context=[f"Test cases to parse: {test_cases_text}"],
    )
    
    find_commands_task = Task(
        description="Based on the extracted test steps, find the appropriate Maestro commands for implementing each step.",
        expected_output="A JSON mapping of test steps to Maestro commands with proper syntax.",
        agent=docs_agent,
        context=[
            "Use the test steps identified in the previous task",
            "For each step, provide the appropriate Maestro command and syntax"
        ],
        dependencies=[parse_test_cases_task]
    )
    
    generate_test_task = Task(
        description=f"Generate a complete Maestro test flow for the {app_name} app that implements the test cases.",
        expected_output="A YAML-formatted Maestro flow file that can be executed to test the app.",
        agent=test_gen_agent,
        context=[
            f"App name: {app_name}",
            "Use the test steps and commands from previous tasks",
            "Create a complete test flow that follows the test cases",
            "Include appropriate assertions to verify app behavior"
        ],
        dependencies=[find_commands_task]
    )
    
    validate_test_task = Task(
        description="Validate the generated test flow for correctness and adherence to Maestro best practices.",
        expected_output="A validated and possibly improved test flow file, ready for execution.",
        agent=validation_agent,
        context=[
            "Check for missing required elements like appId",
            "Ensure all commands have proper syntax",
            "Verify logical flow of the test steps"
        ],
        dependencies=[generate_test_task]
    )
    
    return [parse_test_cases_task, find_commands_task, generate_test_task, validate_test_task]

## Crew Setup
The crew orchestrates the agents and tasks to work together.

In [ ]:
# Step 9: Create the Crew
def create_maestro_crew(app_name, test_cases_text):
    tasks = create_tasks(app_name, test_cases_text)
    
    crew = Crew(
        agents=[parser_agent, docs_agent, test_gen_agent, validation_agent],
        tasks=tasks,
        verbose=2,
        process=Process.sequential  # Tasks need to be executed in order
    )
    
    return crew


## Main Test Generation Function
This function drives the entire process from screenshots to test script.

In [ ]:
# Step 10: Main function to use the crew
def generate_maestro_tests_from_text(app_name, test_cases_text):
    display(Markdown(f"## Generating Maestro tests for {app_name}"))
    display(Markdown("### Written Test Cases:"))
    display(Markdown(f"```\n{test_cases_text}\n```"))
    
    display(Markdown("### Starting CrewAI workflow"))
    crew = create_maestro_crew(app_name, test_cases_text)
    result = crew.kickoff()
    
    # Save the test flow to a file
    test_flow = result
    yaml_filename = f"{app_name}_test_flow.yaml"
    with open(yaml_filename, "w") as f:
        f.write(test_flow)
    
    display(Markdown(f"### Generated Test Flow for {app_name}"))
    display_code(test_flow, "yaml")
    
    display(Markdown(f"Test flow saved to `{yaml_filename}`"))
    return test_flow

## Interactive Interface

In [ ]:
# Create a widget-based interface for entering test cases
def create_test_case_interface():
    # Create widgets
    app_name_input = widgets.Text(
        value='SampleLoginApp',
        placeholder='Enter the app name',
        description='App Name:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '50%'}
    )
    
    test_cases_input = widgets.Textarea(
        value='',
        placeholder='Enter your test cases here...',
        description='Test Cases:',
        disabled=False,
        layout={'width': '100%', 'height': '300px'}
    )
    
    # Sample button to load example test cases
    def load_sample(_):
        test_cases_input.value = """# Login Test Cases for SampleLoginApp

1. Verify user can login with valid credentials
   - Enter valid username (user@example.com)
   - Enter valid password (password123)
   - Tap on Login button
   - Verify user is redirected to the home screen

2. Verify validation for empty username
   - Leave username field empty
   - Enter password (password123)
   - Tap on Login button
   - Verify error message "Username is required" is displayed

3. Verify validation for empty password
   - Enter username (user@example.com)
   - Leave password field empty
   - Tap on Login button
   - Verify error message "Password is required" is displayed

4. Verify "Remember Me" functionality
   - Enter username (user@example.com)
   - Enter password (password123)
   - Tap on "Remember Me" checkbox
   - Tap on Login button
   - Logout from the app
   - Relaunch the app
   - Verify username is pre-filled
"""
    
    sample_button = widgets.Button(
        description='Load Sample Test Cases',
        disabled=False,
        button_style='info',
        tooltip='Click to load sample test cases',
        icon='check'
    )
    sample_button.on_click(load_sample)
    
    # Create a button to generate tests
    output = widgets.Output()
    
    def on_button_clicked(_):
        with output:
            output.clear_output()
            if not app_name_input.value or not test_cases_input.value:
                print("Please enter both app name and test cases.")
                return
                
            print("Generating Maestro test scripts... This may take a minute.")
            generate_maestro_tests_from_text(app_name_input.value, test_cases_input.value)
    
    button = widgets.Button(
        description='Generate Maestro Tests',
        disabled=False,
        button_style='success',
        tooltip='Click to generate tests',
        icon='play'
    )
    button.on_click(on_button_clicked)
    
    # Create the layout
    display(Markdown("## Interactive Test Case Converter"))
    display(Markdown("Enter your app name and test cases below, then click 'Generate Maestro Tests'."))
    display(app_name_input)
    display(sample_button)
    display(test_cases_input)
    display(button)
    display(output)

# Display the interface
create_test_case_interface()

## Maestro Reference


In [ ]:
## Maestro Reference
# Helper cell to show Maestro documentation
display(Markdown("## Maestro Documentation Reference"))
display(Markdown("""
Common Maestro commands:

```yaml
# Launch an app
- launchApp:
    appId: com.example.app

# Tap on an element by text
- tapOn: "Login"

# Input text
- tapOn: "Username"
- inputText: "user@example.com"

# Assert an element is visible
- assertVisible: "Welcome"

# Wait before next action
- wait: 2000  # milliseconds

# Scroll up/down
- scroll
- scrollUp

# Press device back button
- pressBack
```

For a complete reference, see [Maestro Documentation](https://maestro.mobile.dev/api-reference/commands)
"""))

## Next Steps
display(Markdown("## Next Steps"))
display(Markdown("""
1. To run the generated test, install Maestro CLI: `curl -Ls "https://get.maestro.mobile.dev" | bash`
2. The generated YAML is automatically saved to a file named `[AppName]_test_flow.yaml`
3. Run the test with: `maestro test [AppName]_test_flow.yaml`

For more information about Maestro, visit [maestro.mobile.dev](https://maestro.mobile.dev)
"""))